In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

# Beam Class Regression Tests
----

In [3]:
from distgen import Generator
input_file = "../examples/data/rad.gaussian.in.json"
gen = Generator(input_file,verbose=0)
beam = gen.beam()

# Beam weights must be normalized:

$\sum_i w_i = 1$

In [4]:
sum_weights = np.sum(beam['w'])
err = sum_weights-1
assert err.magnitude==0, 'Error: weights unnormalized, sum = !'+str(sum_weights)

In [11]:
from distgen.beam import Beam2
b = Beam2(beam)

In [12]:
print(b['xp'])

[0.08116636459524436 0.23318482862182346 -0.3366402726227139 ... -0.41865547502270983 -0.16536994148423143 -0.03399492624544701] dimensionless
